In [14]:
#================================================================
# ▶모듈불러오기
#================================================================
# 시스템
import os
import sys
import time
import random

# 데이터분석
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 평가지표
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

# 데이터분할
from sklearn.model_selection import train_test_split

# 토치
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# 모델
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import RobertaTokenizer, RobertaForTokenClassification,RobertaTokenizerFast
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# 기타
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [2]:
#===============================================================================
# ▶ 전역설정
#===============================================================================
TOKEN = '---COPY YOUR TOKEN---'
MODEL_NAME = "EbanLee/kobart-summary-v3"

MODEL = 'kobart'
VERSION = 'STEP-0'

MAX_LEN = 512
# MAX_LEN = 128
# MAX_LEN = 250 
BATCH_SIZE =4
EPOCHS = 1
LEARNING_RATE = 5e-5
# LEARNING_RATE = 1e-3
SEED = 2025
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# #===============================================================================
# # ▶ 허깅 페이스 로그인
# #===============================================================================
# from huggingface_hub import login
# # TOKEN = open('TOKEN.TXT').read().strip()
# login(TOKEN)

In [4]:
#===============================================================================
# ▶ 모델 & 토크나이저 불러오기
#===============================================================================
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)
model = BartForConditionalGeneration.from_pretrained(MODEL_NAME)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [11]:
#===============================================================================
# ▶ 토크나이저 세팅
#===============================================================================
input_text = """Abstract
We introduce a new language representation model called BERT, which stands for
Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers.
"""
inputs = tokenizer(input_text, return_tensors="pt", padding="max_length", truncation=True, max_length=MAX_LEN)

In [12]:
#=============================================================================
# 요약 모델 파라미터 설정 
#============================================================================= 
summary_text_ids = \
model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                bos_token_id=model.config.bos_token_id,
                eos_token_id=model.config.eos_token_id,
                length_penalty=1.0,
                max_length=300,
                min_length=12,
                num_beams=6,
                repetition_penalty=1.5,
                no_repeat_ngram_size=15,
                )

In [13]:
#=============================================================================
# 결과확인 
#============================================================================= 
summary_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
print(f'원문\n{input_text}')
print(f'원문길이 :{len(input_text)}')
print(f'요약문\n{summary_text}')
print(f'요약문길이 :{len(summary_text)}')


원문
Abstract
We introduce a new language representation model called BERT, which stands for
Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers.

원문길이 :390
요약문
We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers.
요약문길이 :135
